In [3]:
import sys
!{sys.executable} -m pip install --user twitter
#from tweepy.streaming import streamListener
#from tweepy import OAuthHandler
#from tweepy import stream
from twitter import *
#from tkinter import *

ACCESS_TOKEN = "988216913284255744-qBZJ3IAkrjBAPLaMRMjhkQrLtKIzn31"
ACCESS_TOKEN_SECRET = "WpcZY8dTZAQVWEJkZBlS8hMQaR9ckqOs8ErFML2x2bVAk"
CONSUMER_KEY = "sRKZDEri6WsOhDmJmxxbnLvyC"
CONSUMER_SECRET = "OJO5n71PvgNE6B6JLePAlRTyOlDoTFFpWziUEE4stu0pbNNLHo"

def getFollowers(user, degree): #recursive function that access followers
    followersDict = {}
    if degree == 3:
        return 0
    x = t.followers.list(screen_name=user, count = 200) #dictionary containing list of followers  
    while True:
        for i in range(0,len(x['users'])): #iterate through list of followers
                follower = x['users'][i]['screen_name'] #access follower username
                followersDict[follower] = x['users'][i]['followers_count']
                print(x['users'][i])
                if not x['users'][i]['protected']: #recursively get follower's followers if account is not private
                    getFollowers(follower, degree + 1)
        cursor = x['next_cursor'] #get next page
        if cursor == 0: #no next page
            break
        x = t.followers.list(screen_name=user, cursor = cursor, count = 200) #turn to next page


# Put in token, token_key, con_secret, con_secret_key
t = Twitter(
    auth=OAuth('988216913284255744-qBZJ3IAkrjBAPLaMRMjhkQrLtKIzn31', 'WpcZY8dTZAQVWEJkZBlS8hMQaR9ckqOs8ErFML2x2bVAk',
               'sRKZDEri6WsOhDmJmxxbnLvyC', 'OJO5n71PvgNE6B6JLePAlRTyOlDoTFFpWziUEE4stu0pbNNLHo'))

getFollowers("dijkstra_edsger", 1)






            



You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
{'id': 2272333778, 'id_str': '2272333778', 'name': 'Francis Schickel', 'screen_name': 'FrancisSchickel', 'location': '', 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': True, 'followers_count': 84, 'friends_count': 96, 'listed_count': 0, 'created_at': 'Thu Jan 02 01:55:57 +0000 2014', 'favourites_count': 34, 'utc_offset': -14400, 'time_zone': 'Eastern Time (US & Canada)', 'geo_enabled': False, 'verified': False, 'statuses_count': 11, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_image_url': 'http:/

{'id': 1167288842, 'id_str': '1167288842', 'name': 'Nathan Reider', 'screen_name': 'night_reider', 'location': '', 'description': 'we cannot think of being acceptable to others until we have first proven acceptable to ourselves', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 312, 'friends_count': 310, 'listed_count': 0, 'created_at': 'Sun Feb 10 23:12:27 +0000 2013', 'favourites_count': 8599, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 2121, 'lang': 'en', 'status': {'created_at': 'Thu Apr 26 18:26:42 +0000 2018', 'id': 989571487928668166, 'id_str': '989571487928668166', 'text': '@samlangdonnn #cambergang', 'truncated': False, 'entities': {'hashtags': [{'text': 'cambergang', 'indices': [14, 25]}], 'symbols': [], 'user_mentions': [{'screen_name': 'samlangdonnn', 'name': 'childish sambino', 'id': 1970227806, 'id_str': '1970227806', 'indices': [0, 13]}], 'urls': []}, 'source': '<a href="ht

{'id': 3057444305, 'id_str': '3057444305', 'name': '¢αяℓєу ℓєѕℓєу', 'screen_name': 'LesleyCarley', 'location': 'Dallas, GA from Omaha, NE', 'description': '22 Years Young💁🏼\u200d♀️♓️\nA Nebraska Girl Living In Georgia✌🏼\nLiving life with my Angels right next to me 👼🏽 \n👻ITSCARLAAYY🤞', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': True, 'followers_count': 67, 'friends_count': 1019, 'listed_count': 0, 'created_at': 'Mon Feb 23 22:43:42 +0000 2015', 'favourites_count': 27, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 39, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_i

{'id': 1240334539, 'id_str': '1240334539', 'name': 'Ryan Reeves', 'screen_name': 'Ryan_Reeves20', 'location': 'Omaha, NE', 'description': "Self Taught, Self Made // Professional Wheelman // UNL '20", 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 265, 'friends_count': 148, 'listed_count': 1, 'created_at': 'Mon Mar 04 02:38:29 +0000 2013', 'favourites_count': 3173, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 3822, 'lang': 'en', 'status': {'created_at': 'Thu Apr 26 04:55:58 +0000 2018', 'id': 989367457268760576, 'id_str': '989367457268760576', 'text': 'RT @kanyewest: my MAGA hat is signed 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥 https://t.co/DrDHJybS8V', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'kanyewest', 'name': 'KANYE WEST', 'id': 169686021, 'id_str': '169686021', 'indices': [3, 13]}], 'urls': [], 'media': [{'id': 989227143828652032, 'id_

{'id': 470705923, 'id_str': '470705923', 'name': 'Juanito', 'screen_name': 'juanramosxl', 'location': 'San Antonio, TX', 'description': 'Build your own legacy, leave your mark and never look back. Jesus Christ is my lord and savior. Texas A&M San-Antonio. Detective in the making.', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 650, 'friends_count': 463, 'listed_count': 2, 'created_at': 'Sun Jan 22 02:11:53 +0000 2012', 'favourites_count': 5412, 'utc_offset': -14400, 'time_zone': 'Eastern Time (US & Canada)', 'geo_enabled': True, 'verified': False, 'statuses_count': 15710, 'lang': 'en', 'status': {'created_at': 'Thu Apr 26 18:07:10 +0000 2018', 'id': 989566572523114498, 'id_str': '989566572523114498', 'text': '@mreyes34 @steelers I’m saying bro, defense wins championships.', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'mreyes34', 'name': 'Miguel', 'id': 621419770, 'id_str': '621419770'